# Background construction

This notebook examines the background construction method and errors.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as de
import dedalus.extras.plot_tools as plot_tools
import atmospheres as atmos
import parameters as param
import mpi4py.MPI as MPI
import uuid
%matplotlib notebook

In [ ]:
import importlib
importlib.reload(param)

In [ ]:
domain, p_full, a_full = atmos.solve_hydrostatic_pressure(param, np.float64)
p_full, p, a_full, a, heq, N2 = atmos.truncate_background(param, p_full, a_full)

In [ ]:
# Plot coefficients
fig, axes = plt.subplots(1, 1)
axes.semilogy(np.abs(p_full['c']), '-', label='p_full')
axes.semilogy(np.abs(a_full['c']), '-', label='a_full')
axes.semilogy(np.abs(N2['c']), '-', label='N2', zorder=0)
axes.semilogy(np.abs(heq['c']), '-', label='heq', zorder=0)
axes.set_color_cycle(None)
axes.semilogy(np.abs(p['c']), 'o')
axes.semilogy(np.abs(a['c']), 'o')
axes.set_color_cycle(None)
axes.axhline(param.pressure_floor*np.max(np.abs(p_full['c'])), ls='dashed', c='b')
axes.axhline(param.background_floor*np.max(np.abs(a_full['c'])), ls='dashed', c='g')
axes.set_xlabel('Tn')
axes.set_ylabel('cn')
axes.set_title('Background field coefficients')
axes.legend(loc='upper right')

In [ ]:
# Plot coefficient errors
fig, axes = plt.subplots(1, 1)
axes.semilogy(np.abs(p_full['c']-p['c']), '-', label='p_full-p')
axes.semilogy(np.abs(a_full['c']-a['c']), '-', label='a_full-a')
axes.axhline(param.pressure_floor*np.max(np.abs(p_full['c'])), ls='dashed', c='b')
axes.axhline(param.background_floor*np.max(np.abs(a_full['c'])), ls='dashed', c='g')
axes.set_xlabel('Tn')
axes.set_ylabel('cn')
axes.set_title('Background field coefficients')
axes.legend(loc='upper right')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)

# Examine hydrostatic equilibrium residual
z = domain.grid(0)
heq.set_scales(1)
ax1.plot(z, heq['g'], '-r')
ax1.set_xlabel('z')
ax1.set_ylabel('Heq residual')
print('Heq Linf error:', np.max(np.abs(heq['g'])))

# Examine buoyancy frequency residual
N2_ref = param.N2_func(z)
N2.set_scales(1)
ax2.plot(z, N2['g']-N2_ref, '-r')
ax2.set_xlabel('z')
ax2.set_ylabel('N2 residual')
print('N2 Linf error:', np.max(np.abs(N2['g']-N2_ref)))

In [ ]:
pz = p.differentiate('z')
Hp = (-p / pz).evaluate()
cs2 = (param.γ * p * a).evaluate()
ωc = (cs2**0.5 / 2 / Hp).evaluate()

In [ ]:
z = domain.grid(0)
fig, axes = plt.subplots(1, 1)
a.set_scales(1)
p.set_scales(1)
Hp.set_scales(1)
cs2.set_scales(1)
axes.plot(z, a['g'], label='a')
axes.plot(z, p['g'], label='p')
axes.plot(z, Hp['g'], label='Hp')
axes.plot(z, cs2['g'], label='cs2')
axes.axhline(0, c='k')
axes.set_xlabel('z')
axes.set_yscale('log')
axes.legend(loc='upper left')

In [ ]:
fig, axes = plt.subplots(1, 1)
N2.set_scales(1)
ωc.set_scales(1)
axes.plot(z, N2['g']**0.5, label='N')
#axes.plot(z, N2_ref**0.5, '--', label='Nr')
axes.plot(z, ωc['g'], label='ωc')
axes.set_xlabel('z')
axes.legend(loc='upper left')
axes.set_ylim(0, None)